encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
# root_path = '/data/hdr/csi_transformer/pic/'
# print(root_path)
# root_path2 = '/'.join(root_path.split("/")[:-2])
# # sys.path.append(root_path)
# print(root_path2)

In [3]:
# method = METHOD_MITHRAL
method = METHOD_PLUTO
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [4]:
linear_name = 'etl1'
feedback_bits = 256
ncodebooks = 64 # max:64
ncentroids = 256
if method == METHOD_MITHRAL or METHOD_PLUTO:
    ncentroids = 16
train_sam_num = 1000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32

In [5]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'ex_linear1in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'ex_linear2in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    # y_train = 'ex_linear1_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'ex_linear1out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'ex_linear1in_test.npy'
    featurepath_test = 'ex_linear1out_test.npy'
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'ex_linear1_w_f256.npy'
biaspath = 'ex_linear1_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, "etl1")
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [6]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=featurepath_train, bias_path=biaspath, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Pluto
(1024000, 512)
(512,)
<class 'numpy.ndarray'>
learn_pluto activation: <function relu at 0x7f6c17064430>
learn_pluto with N:1024000 D:64 M:512 DQ:64 obj:mse
X_res mse / X mse:  0.005530617
encoded_pluto bias: tensor([-0.0268, -0.0405, -0.1348, -0.0301, -0.0754, -0.0878,  0.0925, -0.1785,
        -0.0925, -0.0966, -0.1247,  0.0067, -0.0831, -0.0332, -0.4592, -0.0810,
        -0.1856, -0.0539, -0.0615, -0.2495,  0.0565, -0.1614, -0.1802, -0.0693,
        -0.1954, -0.0729, -0.0446,  0.1022, -0.2439, -0.1410, -0.1176, -0.0733,
        -0.1257, -0.0485, -0.2143, -0.2450, -0.1673, -0.0336, -0.1063, -0.0794,
        -0.0860, -0.0284, -0.1015, -0.3457, -0.2414, -0.1669, -0.0887, -0.0667,
         0.0249, -0.1095, -0.1175, -0.0603, -0.1833, -0.0470, -0.0740, -0.1681,
        -0.0251, -0.2860, -0.0103, -0.1042, -0.5400, -0.1151, -0.2281,  0.0769,
        -0.0552, -0.0649, -0.0488, -0.0457, -0.1391, -0.0608,  0.0294, -0.1072,
         0.0978, -0.1705, -0.0619, -0.0858, -0.06

KeyboardInterrupt: 

In [ ]:
# type(est3)

In [ ]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul# + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [ ]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1])
print("y_out_last_re.shape: ", y_out_last_re.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.30822068 -0.30822068 -0.30822068 ... -0.18297607  1.3199594
  -0.30822068]
 [-0.4334653  -0.30822068 -0.30822068 ... -0.05773144  1.0694702
  -0.30822068]
 [-0.30822068 -0.30822068 -0.18297607 ... -0.18297607  1.0694702
  -0.30822068]
 ...
 [-0.30822068 -0.18297607 -0.05773144 ... -0.05773144  0.56849164
  -0.18297607]
 [-0.30822068 -0.30822068 -0.18297607 ... -0.05773144  0.8189809
  -0.30822068]
 [-0.18297607 -0.18297607  0.06751318 ...  0.06751318  0.69373626
   0.06751318]]
y_out_last.shape:  (1024000, 512)
y_out_last_re.shape:  (1000, 32, 32, 512)
